In [32]:
!pip install scikit-optimize

     ---------------------------------------- 0.0/100.3 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/100.3 kB ? eta -:--:--
     ------------------------------------ 100.3/100.3 kB 968.0 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [116]:
import pandas as pd
import numpy as np
import random
import os
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything()

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from skopt import BayesSearchCV

import warnings
warnings.filterwarnings('ignore')
from bayes_opt import BayesianOptimization

In [97]:
train = pd.read_csv('preprocessed_csv/train_preprocessed.csv')
test = pd.read_csv('preprocessed_csv/test_preprocessed.csv')

In [98]:
train[['com_reg_ver_win_rate', 'ver_win_rate_x', 'ver_win_ratio_per_bu']] = train[['com_reg_ver_win_rate', 'ver_win_rate_x', 'ver_win_ratio_per_bu']].replace(np.nan, 0)
test[['com_reg_ver_win_rate', 'ver_win_rate_x', 'ver_win_ratio_per_bu']] = test[['com_reg_ver_win_rate', 'ver_win_rate_x', 'ver_win_ratio_per_bu']].replace(np.nan, 0)

In [100]:
def encode_categorical_variables(train, test):
    categorical_columns = train.select_dtypes(include=['object']).columns.tolist()
    
    label_encoders = {}
    
    for col in categorical_columns:
        le = LabelEncoder()
        le = le.fit(train[col])
        train[col] = le.transform(train[col])
        
        for label in np.unique(test[col]):
            if label not in le.classes_:
                le.classes_ = np.append(le.classes_, label)
        test[col] = le.transform(test[col])
    
    return train, test

train, test = encode_categorical_variables(train, test)

In [102]:
X = train.drop(columns='is_converted', axis=1)
y = train['is_converted']

# RandomForestClassifier

In [123]:
def stratified_kfold_score(clf, X, y, n_fold):
    X, y = X.values, y.values
    strat_kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    roc_auc_list = []

    for train_index, test_index in strat_kfold.split(X, y):
        x_train_fold, x_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        clf.fit(x_train_fold, y_train_fold)
        preds = clf.predict(x_test_fold)
        roc_auc_test = roc_auc_score(y_test_fold, preds)
        roc_auc_list.append(roc_auc_test)

    return np.mean(roc_auc_list)

In [124]:
def bo_params_rf(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    params = {
        'n_estimators' : int(n_estimators),
        'max_depth' : int(max_depth),
        'min_samples_split' : int(min_samples_split),
        'min_samples_leaf' : int(min_samples_leaf),
        'random_state': 42
    }
    clf = DecisionTreeClassifier(**params)
    return stratified_kfold_score(clf, X, y, 5)

In [126]:
rf_bo = BayesianOptimization(
    f=bo_params_rf,
    pbounds={
        'n_estimators': (100, 1000),
        'max_depth': (10, 100),
        'min_samples_split': (2, 100),
        'min_samples_leaf': (1, 50)
    },
    random_state=42,
)

rf_bo.maximize(
    n_iter=50,
    init_points=2,
)

print(rf_bo.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9761    | 43.71     | 47.59     | 73.74     | 638.8     |
| 2         | 0.9817    | 24.04     | 8.644     | 7.692     | 879.6     |
| 3         | 0.9806    | 24.02     | 14.21     | 11.79     | 878.6     |
| 4         | 0.985     | 26.45     | 1.937     | 14.74     | 902.4     |
| 5         | 0.9818    | 10.4      | 2.978     | 47.71     | 416.9     |
| 6         | 0.9767    | 97.37     | 35.77     | 99.4      | 436.5     |
| 7         | 0.9839    | 24.41     | 1.121     | 19.32     | 900.2     |
| 8         | 0.9814    | 47.92     | 10.63     | 13.45     | 921.9     |
| 9         | 0.9821    | 19.81     | 9.215     | 7.652     | 914.5     |
| 10        | 0.9821    | 45.82     | 7.788     | 15.36     | 894.6     |
| 11        | 0.9811    | 11.91     | 7.4       | 23.09     | 385.6     |
| 12        | 0.9824    | 10.5      | 

# XGB

In [130]:
def stratified_kfold_score(xgb, X, y, n_fold):
    X, y = X.values, y.values
    strat_kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    roc_auc_list = []

    for train_index, test_index in strat_kfold.split(X, y):
        x_train_fold, x_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        xgb.fit(x_train_fold, y_train_fold)
        preds = xgb.predict(x_test_fold)
        roc_auc_test = roc_auc_score(y_test_fold, preds)
        roc_auc_list.append(roc_auc_test)

    return np.mean(roc_auc_list)

In [131]:
def bo_params_xgb(n_estimators, max_depth, learning_rate, min_child_weight, subsample, colsample_bytree, gamma):
    params = {
        'n_estimators' : int(n_estimators),
        'max_depth' : int(max_depth),
        'learning_rate' : learning_rate,
        'min_child_weight' : int(min_child_weight),
        'subsample' : subsample,
        'colsample_bytree' : colsample_bytree,
        'gamma' : gamma,
        'random_state': 42
    }
    xgb = XGBClassifier(**params)
    return stratified_kfold_score(xgb, X, y, 5)

In [132]:
xgb_bo = BayesianOptimization(
    f=bo_params_xgb,
    pbounds={
        'n_estimators': (100, 1000),
        'max_depth': (3, 10),
        'learning_rate' : (0.01, 0.3),
        'min_child_weight' : (1, 10),
        'subsample' : (0.5, 1.0),
        'colsample_bytree' : (0.5, 1.0),
        'gamma' : (0, 5)
    },
    random_state=42,
)

xgb_bo.maximize(
    n_iter=50,
    init_points=2,
)

print(xgb_bo.max)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.9885    | 0.6873    | 4.754     | 0.2223    | 7.191     | 2.404     | 240.4     | 0.529     |
| 2         | 0.9883    | 0.9331    | 3.006     | 0.2153    | 3.144     | 9.729     | 849.2     | 0.6062    |
| 3         | 0.9884    | 0.9813    | 4.18      | 0.2118    | 5.863     | 2.56      | 240.8     | 0.6251    |
| 4         | 0.9862    | 0.7316    | 4.132     | 0.01213   | 7.498     | 3.51      | 240.7     | 0.8691    |
| 5         | 0.988     | 0.7128    | 1.704     | 0.2724    | 6.308     | 9.484     | 367.5     | 0.8402    |
| 6         | 0.9875    | 0.9993    | 0.2558    | 0.2369    | 8.196     | 8.131     | 414.4     | 0.7322    |
| 7         | 0.9877    | 0.778     | 4.519     | 0.2019    | 4.614     | 9.695     | 456.5     | 0.5138    |
| 8       

| 45        | 0.9863    | 0.7275    | 3.345     | 0.03186   | 8.745     | 2.843     | 101.5     | 0.6912    |
| 46        | 0.9894    | 0.8935    | 4.451     | 0.1475    | 6.713     | 1.595     | 239.9     | 0.846     |
| 47        | 0.9888    | 0.9163    | 4.467     | 0.2813    | 4.383     | 1.977     | 240.6     | 0.6545    |
| 48        | 0.9891    | 0.7652    | 3.152     | 0.297     | 4.895     | 6.994     | 844.5     | 0.795     |
| 49        | 0.9893    | 0.854     | 3.922     | 0.1702    | 6.744     | 6.088     | 586.3     | 0.8601    |
| 50        | 0.989     | 0.9675    | 4.783     | 0.1357    | 4.201     | 9.105     | 900.4     | 0.7572    |
| 51        | 0.9886    | 0.8639    | 2.482     | 0.156     | 8.509     | 2.655     | 184.8     | 0.6394    |
| 52        | 0.9885    | 0.6374    | 3.35      | 0.1933    | 4.483     | 3.83      | 398.4     | 0.6337    |
{'target': 0.9894828211065727, 'params': {'colsample_bytree': 0.8497054793943353, 'gamma': 3.935081236491979, 'learning_